In [1]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

with tf.name_scope('HiddenLayers'):
    model.add(Dense(units=128, input_shape=[4]))
    model.add(Dense(units=128, input_shape=[4]))
    
model.add(Dense(units=128, input_shape=[4]))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               640       
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
Total params: 33,664
Trainable params: 33,664
Non-trainable params: 0
_________________________________________________________________


In [2]:
import numpy as np
from IPython.display import display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [3]:
from keras.models import Model
from keras.layers import Input, Dense

input = Input(shape=[4])

with tf.name_scope('HiddenLayersFunctional'):
    h1 = Dense(units=128, input_shape=[4])(input)
    h2 = Dense(units=128)(h1)
    h3 = Dense(units=128)(h2)
p = Dense(units=1)(h3)

model = Model(inputs=input, outputs=p)
model.compile(loss='binary_crossentropy', optimizer='adam')

In [4]:
show_graph(sess.graph)